In [28]:
import optuna
# import wandb
import logging
import sys
import os
import tensorflow as tf
import json
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Conv1D, Dense, LSTM, Dropout, Bidirectional, MaxPooling1D, GlobalAveragePooling1D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import l2
from tensorflow.keras.layers import Concatenate, BatchNormalization, MultiHeadAttention, LayerNormalization
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report
from sklearn.metrics import confusion_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split
from tensorflow.keras.callbacks import EarlyStopping, Callback
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from sklearn.feature_extraction.text import TfidfVectorizer


In [29]:
url_data = 'https://raw.githubusercontent.com/TeodorRusKvi/Tekstanalyse/main/git_NLP_data/'

# Last inn 'y_train_LSTM' fra en CSV-fil
y_df = pd.read_csv(url_data+'y_liberal.csv')
# Konverter hele DataFrame til et NumPy array
y = y_df.to_numpy()

# Last inn 'y_train_LSTM' fra en CSV-fil
embeddings_GloVe = pd.read_csv(url_data+'embeddings_glove.csv')
# Konverter hele DataFrame til et NumPy array
embeddings_GloVe = embeddings_GloVe.to_numpy()

# Last inn 'X_train_LSTM' fra en CSV-fil
X_df = pd.read_csv(url_data+'new_df.csv')

In [43]:
X_1 = X_df['without_stopwords'].astype(str)

In [39]:
X= X_df['without_stopwords'].to_numpy().flatten().astype(str)

In [40]:
all_texts_length = X_df['without_stopwords'].apply(lambda x: len(x.split()))
# Now, let's analyze the distribution of these sequence lengths
all_texts_length.describe()

AttributeError: 'float' object has no attribute 'split'

In [41]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(X)

# Get feature names to later identify words by index
feature_names = vectorizer.get_feature_names_out()

# Calculate the average TF-IDF score for each word across all documents
avg_scores = np.mean(X.toarray(), axis=0)

# Set a threshold, for example, the mean of average scores
threshold = np.mean(avg_scores)

# Filter words that have a score above the threshold
important_words = [feature_names[i] for i in range(len(feature_names)) if avg_scores[i] > threshold]

In [44]:
# Filter corpus to keep only important words
filtered_texts = []
for doc in X_1:
    tokens = doc.split()
    filtered_tokens = [token for token in tokens if token in important_words]
    filtered_texts.append(' '.join(filtered_tokens))

# Tokenize texts
tokenizer = Tokenizer()
tokenizer.fit_on_texts(filtered_texts)
sequences = tokenizer.texts_to_sequences(filtered_texts)

# Pad sequences to have the same length
max_seq_length = max(len(seq) for seq in sequences)

In [46]:
#Creating a word index of the words from the tokenizer 
word_index = tokenizer.word_index
print(f'Found {len(word_index)} unique tokens.\n\nFirst 10 is listen below:')
print(dict(list(word_index.items())[0:10]))

Found 3325 unique tokens.

First 10 is listen below:
{'the': 1, 'be': 2, 'to': 3, 'of': 4, 'and': 5, 'in': 6, 'nan': 7, 'that': 8, 'for': 9, 'it': 10}


In [47]:
# Defining pre-processing hyperparameters for the networks
max_len = 100
trunc_type = "post"
padding_type = "post"
vocab_size = len(word_index)
# This is fixed.
embedding_dim = 100
EPOCHS=20
BATCH_SIZE = 32
num_classes = 1

# Padding the sequences to keep the lengths uniform
X = pad_sequences(sequences, maxlen=max_len, padding=padding_type, truncating=trunc_type)
# print('Shape of data tensor:', X_tensorflow.shape)

In [48]:
# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=42, stratify=y_test)

In [49]:
class ModelConfig:
    def __init__(self, max_len, num_classes, embeddings_GloVe):
        self.max_len = max_len
        self.num_classes = num_classes
        self.embeddings_GloVe = embeddings_GloVe


# Create a configuration object
config = ModelConfig(max_len=max_len, num_classes=num_classes, embeddings_GloVe=embeddings_GloVe)

In [50]:
def CNN_LSTM_sequential(params, config):
    input_layer = Input(shape=(config.max_len,), dtype='int32')
    
    # Use config object for fixed parameters such as embeddings
    embedding = Embedding(input_dim=config.embeddings_GloVe.shape[0],
                          output_dim=config.embeddings_GloVe.shape[1],
                          weights=[config.embeddings_GloVe],
                          trainable=False)(input_layer)
    
    # Use params dictionary for dynamic hyperparameters
    dropout = Dropout(params['dropout_rate'])(embedding)

    conv = Conv1D(filters=params['conv_filters'], kernel_size=1, activation='relu')(dropout)
    conv = BatchNormalization()(conv)

    lstm = Bidirectional(LSTM(params['lstm_units'], return_sequences=True, dropout=0.006, recurrent_dropout=0.1))(conv)
    lstm = LayerNormalization()(lstm)
    
    num_heads = 8
    attention_layer = MultiHeadAttention(num_heads=num_heads, key_dim=config.embeddings_GloVe.shape[1] // num_heads, dropout=0.1)
    attention_output = attention_layer(query=lstm, key=lstm, value=lstm)
    attention_output = LayerNormalization()(attention_output)

    dense = Dense(params['dense_2_units'], activation='relu')(attention_output)
    dense = BatchNormalization()(dense)
    output = Dense(config.num_classes, activation='sigmoid')(dense)
    
    model = Model(inputs=input_layer, outputs=output)
    model.compile(loss='binary_crossentropy', optimizer=Adam(params['learning_rate']), metrics=['accuracy'])
    
    return model

In [51]:
# Loading the params from CNN-LSTM from the saved JSON-fil
with open(r'C:\Users\bugat\Prosjekter\Tekstanalyse\git_NLP\Tekstanalyse\git_NLP_Notebooks\best_trial_length_100.json', 'r') as f:
    data = json.load(f)
    params = data['params']

In [52]:
params

{'lstm_units': 140,
 'dense_2_units': 150,
 'dropout_rate': 0.42803898610506674,
 'learning_rate': 0.000922823163674921,
 'conv_filters': 52}

In [53]:
best_model = CNN_LSTM_sequential(params, config)

best_model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=20, batch_size=32, verbose=1)
# Retrain on the full training data

# Evaluate on the test data
loss_1, accuracy_1 = best_model.evaluate(X_test, y_test, verbose=1)
print(f"Test Accuracy: {accuracy_1}")



Epoch 1/20


282/282 [==============================] - 120s 389ms/step - loss: 0.6583 - accuracy: 0.6364 - val_loss: 0.6858 - val_accuracy: 0.6473
Epoch 2/20
282/282 [==============================] - 107s 380ms/step - loss: 0.6372 - accuracy: 0.6472 - val_loss: 0.6252 - val_accuracy: 0.6592
Epoch 3/20
282/282 [==============================] - 108s 382ms/step - loss: 0.6260 - accuracy: 0.6591 - val_loss: 0.6115 - val_accuracy: 0.6714
Epoch 4/20
282/282 [==============================] - 110s 389ms/step - loss: 0.6182 - accuracy: 0.6640 - val_loss: 0.6878 - val_accuracy: 0.6539
Epoch 5/20
282/282 [==============================] - 112s 399ms/step - loss: 0.6046 - accuracy: 0.6822 - val_loss: 0.5845 - val_accuracy: 0.7099
Epoch 6/20
282/282 [==============================] - 107s 381ms/step - loss: 0.5949 - accuracy: 0.6871 - val_loss: 0.5831 - val_accuracy: 0.7037
Epoch 7/20
282/282 [==============================] - 108s 382ms/step - loss: 0.5889 - accuracy: 0.6938 - val_loss: 0.57

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc}, Test Loss: {test_loss}')

In [ ]:
plt.figure(figsize=(9,7))
plt.title('Accuracy score')
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.legend(['accuracy', 'val_accuracy'])
plt.show()
plt.figure(figsize=(9,7))
plt.title('Loss value')
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(['loss', 'val_loss'])
plt.show()